In [ ]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import ParameterGrid

# Download file with AMSR2 and SIT data

In [ ]:
!wget ftp://ftp.nersc.no/pub/ml_course/amsr2_201501311200.npz

# Load data into notebook

### Features

AMSR2 has 14 bands: observations of surface at 7 passive microwave frequencies in two polarizations.

### Targets

Sea ice type shows "age" of ice. 1 - open water, 2 - first-year ice, 3 - multi-year ice (0 - invalid, 4 - uncertain).

### Data is loaded into a dictionary `d`

The data is stored as a collection of 2D arrays with names 'h06', 'v06', etc. and 'sit'.


In [ ]:
# load data into dictionary
d = dict(**np.load('amsr2_201501311200.npz'))
# names of bands in the dictionary:
print(d.keys())
# size of 2D arrays
print(d['h06'].shape, d['sit'].shape)

In [ ]:
# select random bands to show maps:
b0 = 'sit' # sea ice type
b1 = 'h06' # Brightness temperature at 6  GHz at horizontal polarization
b2 = 'v06' # Brightness temperature at 6  GHz at vertical polarization
b3 = 'v89' # Brightness temperature at 89 GHz at vertical polarization

# plot maps of selected bands
fig, ax = plt.subplots(1,4, figsize=(20,5))
for i,b in enumerate([b0, b1, b2, b3]):
    img = ax[i].imshow(d[b], cmap='jet', interpolation='nearest')
    plt.colorbar(img, ax=ax[i], shrink=0.3)
    ax[i].set_title(b)
plt.show()

# Convert 2D maps into 1D vectors and compare

### Boolean indexing

`mask` is a 2D boolean array with True where data is valid (0 < sit < 4) and False where data is invalid (land and uncertain pixels)

`d['sit'][mask]` is a 1D vector with sea ice type for valid pixels only

`d['h06'][mask]` is a 1D vector with AMSR2 measurements in valid pixels

### Comparison of bands

Some bands are very correlated (e.g. 'h06' and 'h07') and it is very difficult to distinguish ice types.
Other bands are less correlated(e.g. 'h06' and 'v89'). The scatter plots compare values of the pixels in different bands, colored by ice type. 

In [ ]:
# 2D mask of valid pixels
mask = (d['sit'] > 0) * (d['sit'] < 4)

# 1D vectors of SIT and AMSR2 measurements at the selectef bands
sit_masked = d['sit'][mask]
d1_masked = d[b1][mask]
d2_masked = d[b2][mask]
d3_masked = d[b3][mask]

# scatterplot
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].scatter(d1_masked, d2_masked, 10, sit_masked, cmap='jet', vmin=0, vmax=4)
ax[0].set_xlabel(b1)
ax[0].set_ylabel(b2)
ax[1].scatter(d1_masked, d3_masked, 10, sit_masked, cmap='jet', vmin=0, vmax=4)
ax[1].set_xlabel(b1)
ax[1].set_ylabel(b3)
plt.show()

# Create training and testing datasets

### Select bands

Only two bands are selected to become features in our first classification. Just for illustration.

### Create arrays of features and targets

2D array of features (N_points x N_features) and 1D array of targets (N_points) are created from valid observations. (`mask` is used).

### Split arrays of features and targets into to sub-samples for training and testing

In [ ]:
# set bands that are used as features
bands = ['h06', 'v06']
# create 2D array of features
X = np.array([d[b][mask] for b in bands]).T
# create 1D vector of targets
Y = d['sit'][mask]
print(X.shape, Y.shape)
# split features and targets by ratio 1/5
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

# Perform Random Forest Training and Classification

In [ ]:
# create a Random Forest classifier with default options
clf = RandomForestClassifier()
# train the classifier
# Yes, MACHINE LEARNING is actually happening here:
clf.fit(X_train,Y_train)
# test the trained classifier on independent data
Y_pred = clf.predict(X_test)
# compute accuracy of the classifier
print(accuracy_score(Y_test, Y_pred))

# Grid search

### 1. Why searching?
The `RandomForestClassifier` has a lot of parameters (e.g., n_estimators, criterion, max_features, etc.).
They have default values. You can see these values in the next cell.

But what if these values are not optimal?

Then we have to search for better parameters. For example in a "Grid Search".

### 2. Define space for searching

We can iterate on all parameters with all possible values but that will take infinite amount of time.
We should choose which parameters we want to test and what are the realistic values to test.
This is defined in `param_space` dictionary.

Then a grid of all combinations of parameters is created (2 parameters, 2 values each ==> 2^3 = 4 combinations).

### 3. Search

Finally, we loop over these combinations, create, train and test the classifier and keep only the best one.


In [ ]:
RandomForestClassifier?

In [ ]:
# define space for searching
param_space = dict(
    max_features = ['sqrt', 'log2'],
    n_estimators = [2, 200],
)

# create grid with all combinations of parameters and values
param_grid = ParameterGrid(param_space)

# loop over the combinations
best_score = 0
best_clf = None
for params in param_grid:
    # create a sample classifier and train
    clf = RandomForestClassifier(**params)
    clf.fit(X_train,Y_train)
    # use the classifier and evaluate accuracy
    Y_pred = clf.predict(X_test)
    score = accuracy_score(Y_test, Y_pred)
    print(score, params)
    # keep the best classifier
    if score > best_score:
        best_score = score
        best_clf = clf
print('Best score:', best_score)        

In [ ]:
# use the best classifier and plot a "Confusion matrix"
Y_best = best_clf.predict(X_test)
plt.hist2d(Y_test, Y_best, 3)
plt.xlabel('True')
plt.ylabel('Predicted')
plt.colorbar()
plt.show()

# which features are more important in classification
print('Feature importance:', list(zip(*[bands, best_clf.feature_importances_])))

# Reconstruct a map with ice type

In [ ]:
# make a new mask that also includes 'uncertain pixels'.
mask_new = d['sit'] > 0

# create 2D array of features from all valid pixels
X_new = np.array([d[b][mask_new] for b in bands]).T

# use the best classifier to predict targets
Y_new = best_clf.predict(X_new)

# create an empty 2D array with sea ice type map
sit_new = np.zeros_like(d['sit'])

# insert the predicted targets into the valid pixels
sit_new[mask_new] = Y_new

# show maps of initial and new ice types
fig, ax = plt.subplots(1,2, figsize=(14,7))
ax[0].imshow(d['sit'], clim=[0,4], cmap='jet', interpolation='nearest')
ax[1].imshow(sit_new, clim=[0,4], cmap='jet', interpolation='nearest')
plt.show()

# Tasks for individual work

### 1. Improve classification accuracy

Hints:    
* Select other features
* Extend search space by other parameters and other values

### 2. Find the best features

Hints:
* Check feature importance and choose only few features

### 3. Overfit

Find such a configuration when you clearly overfit the model.

Hints:
* Split training / testing data differently
* Test other parameters